In [1]:
import os
import tensorflow as tf
import random

In [2]:
Scaled_Path='C:\\Tensorflow\\Rescaled_DB'
hanDB_train='\\HanDB_train\\'
file_list=os.listdir(Scaled_Path+hanDB_train)

for i in range(len(file_list))
    file_path=Scaled_Path+hanDB_train+file_list[i]
    f = open(file_path,'rb')
    file_header = f.read(8)
    
    code=f.read(2)
    code_numb=int.from_bytes(code, byteorder='big')


SyntaxError: invalid syntax (<ipython-input-2-58e8e07c9c1b>, line 5)

In [4]:
def read_hgu1(file_path):
        
    f = open(file_path, 'rb')
    file_header = f.read(8)
    
    # Read Images

    # read code & check EoF
    code = f.read(2)
    width=f.read(1)[0]
    height = f.read(1)[0]
    reserved = f.read(2)
    image_byte=[]
    
    for i in range(height):
        buffer=[]
        for j in range(width):
            data=int.from_bytes(f.read(1), byteorder='big')
            buffer.append(data)
            #print(data, end=' ')
        image_byte.append(buffer)
                           
        # Reading file done
    f.close()
    
    # Read Next file
    return code, width, height, image_byte


In [9]:
Scaled_Path='C:\\Tensorflow\\Rescaled_DB'
hanDB_train='\\HanDB_train\\'
file_list=os.listdir(Scaled_Path+hanDB_train)


file_path=Scaled_Path+hanDB_train+file_list[0]
code,width,height,image_byte=read_hgu1(file_path)

print(int.from_bytes(code, byteorder='big'), width, height)
#print(image_byte)

45217 64 64


In [4]:
def make_TFRecord(file_path,TFRecord_name):
    # read files 
    file_list=os.listdir(file_path)
    examples=[]
    writer = tf.python_io.TFRecordWriter("./"+TFRecord_name+'.tfrecord')
    
    for file_no in range(len(file_list)):
        # file open
        EOF=False
        myfile=file_list[file_no]
        f = open(file_path+ myfile, 'rb')
        
        # Read Header
        file_header = f.read(8) 
        # Read Imges
    
        while EOF==False : 
        # read code & check EoF
            code = f.read(2)
            if (len(code) < 2) :
                EOF=True;
                break
            code_numb=int.from_bytes(code, byteorder='big')
            width=f.read(1)[0]
            height = f.read(1)[0]
            reserved = f.read(2)
            image_bytes=f.read(width*height)
            
            example = tf.train.Example(features=tf.train.Features(feature={'label':
            tf.train.Feature(int64_list=tf.train.Int64List(value=[code_numb])), 'height':
            tf.train.Feature(int64_list=tf.train.Int64List(value=[height])), 'width':
            tf.train.Feature(int64_list=tf.train.Int64List(value=[width])), 'image':
            tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))}
            ))   
            examples.append(example) 
        f.close()
        print(str(file_no)+' '+myfile+" "+ str(code_numb)+" is added to TFRecord.")
        
    random.shuffle(examples) 
    for counts in range(len(examples)):
        writer.write(examples[counts].SerializeToString())
    writer.close()
   


In [7]:
def test_make_TFRecord(file_path):
    # make file list
    file_list=os.listdir(file_path)
    examples=[]
    writer = tf.python_io.TFRecordWriter("./test_record.tfrecord")
    # file open
    EOF=False
    myfile=file_list[0]
    f = open(file_path+ myfile, 'rb')
    file_header = f.read(8) 
    # Read Imges
    while EOF==False : 
        # read code & check EoF
        code = f.read(2)
        if (len(code) < 2) :
            break

        code_numb=int.from_bytes(code,byteorder='big')        
        width=f.read(1)[0]
        height = f.read(1)[0]
        reserved = f.read(2)
        image_bytes=f.read(width*height)
        
        example = tf.train.Example(features=tf.train.Features(feature={'label':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[code_numb])), 'height':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[height])), 'width':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[width])), 'image':
        tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))}
        ))
        # Reading file done
        #print(example)
        examples.append(example) 
        break;
    f.close()
    random.shuffle(examples)    
    for counts in range(len(examples)):
        writer.write(examples[counts].SerializeToString())
    
    writer.close
        #print("File "+str(fine_no)+" is rescaled")
        # Read Next file
    print(myfile+"converted to tfrecord!!")

In [ ]:
origin_path='C:\\Tensorflow\\Rescaled_DB\\'
seri_test='SERI_Test\\'
han_train='HanDB_Train\\'
han_test='HanDB_Test\\'
file_path=origin_path+han_train
make_TFRecord(file_path,"int_HanDB_Train")
#test_make_TFRecord(file_path)

0 r_b0a1.hgu1 45217 is added to TFRecord.
1 r_b0a2.hgu1 45218 is added to TFRecord.
2 r_b0a3.hgu1 45219 is added to TFRecord.
3 r_b0a4.hgu1 45220 is added to TFRecord.
4 r_b0a5.hgu1 45221 is added to TFRecord.
5 r_b0a6.hgu1 45222 is added to TFRecord.
6 r_b0a7.hgu1 45223 is added to TFRecord.
7 r_b0a8.hgu1 45224 is added to TFRecord.
8 r_b0a9.hgu1 45225 is added to TFRecord.
9 r_b0aa.hgu1 45226 is added to TFRecord.
10 r_b0ab.hgu1 45227 is added to TFRecord.
11 r_b0ac.hgu1 45228 is added to TFRecord.
12 r_b0ad.hgu1 45229 is added to TFRecord.
13 r_b0ae.hgu1 45230 is added to TFRecord.
14 r_b0af.hgu1 45231 is added to TFRecord.
15 r_b0b0.hgu1 45232 is added to TFRecord.
16 r_b0b1.hgu1 45233 is added to TFRecord.
17 r_b0b2.hgu1 45234 is added to TFRecord.
18 r_b0b3.hgu1 45235 is added to TFRecord.
19 r_b0b4.hgu1 45236 is added to TFRecord.
20 r_b0b5.hgu1 45237 is added to TFRecord.
21 r_b0b6.hgu1 45238 is added to TFRecord.
22 r_b0b7.hgu1 45239 is added to TFRecord.
23 r_b0b8.hgu1 45240 

192 r_b2a5.hgu1 45733 is added to TFRecord.
193 r_b2a6.hgu1 45734 is added to TFRecord.
194 r_b2a7.hgu1 45735 is added to TFRecord.
195 r_b2a8.hgu1 45736 is added to TFRecord.
196 r_b2a9.hgu1 45737 is added to TFRecord.
197 r_b2aa.hgu1 45738 is added to TFRecord.
198 r_b2ab.hgu1 45739 is added to TFRecord.
199 r_b2ac.hgu1 45740 is added to TFRecord.
200 r_b2ad.hgu1 45741 is added to TFRecord.
201 r_b2ae.hgu1 45742 is added to TFRecord.
202 r_b2af.hgu1 45743 is added to TFRecord.
203 r_b2b0.hgu1 45744 is added to TFRecord.
204 r_b2b1.hgu1 45745 is added to TFRecord.
205 r_b2b2.hgu1 45746 is added to TFRecord.
206 r_b2b3.hgu1 45747 is added to TFRecord.
207 r_b2b4.hgu1 45748 is added to TFRecord.
208 r_b2b5.hgu1 45749 is added to TFRecord.
209 r_b2b6.hgu1 45750 is added to TFRecord.
210 r_b2b7.hgu1 45751 is added to TFRecord.
211 r_b2b8.hgu1 45752 is added to TFRecord.
212 r_b2b9.hgu1 45753 is added to TFRecord.
213 r_b2ba.hgu1 45754 is added to TFRecord.
214 r_b2bb.hgu1 45755 is added t

384 r_b4a9.hgu1 46249 is added to TFRecord.
385 r_b4aa.hgu1 46250 is added to TFRecord.
386 r_b4ab.hgu1 46251 is added to TFRecord.
387 r_b4ac.hgu1 46252 is added to TFRecord.
388 r_b4ad.hgu1 46253 is added to TFRecord.
389 r_b4ae.hgu1 46254 is added to TFRecord.
390 r_b4af.hgu1 46255 is added to TFRecord.
391 r_b4b0.hgu1 46256 is added to TFRecord.
392 r_b4b1.hgu1 46257 is added to TFRecord.
393 r_b4b2.hgu1 46258 is added to TFRecord.
394 r_b4b3.hgu1 46259 is added to TFRecord.
395 r_b4b4.hgu1 46260 is added to TFRecord.
396 r_b4b5.hgu1 46261 is added to TFRecord.
397 r_b4b6.hgu1 46262 is added to TFRecord.
398 r_b4b7.hgu1 46263 is added to TFRecord.
399 r_b4b8.hgu1 46264 is added to TFRecord.
400 r_b4b9.hgu1 46265 is added to TFRecord.
401 r_b4ba.hgu1 46266 is added to TFRecord.
402 r_b4bb.hgu1 46267 is added to TFRecord.
403 r_b4bc.hgu1 46268 is added to TFRecord.
404 r_b4bd.hgu1 46269 is added to TFRecord.
405 r_b4be.hgu1 46270 is added to TFRecord.
406 r_b4bf.hgu1 46271 is added t

579 r_b6b0.hgu1 46768 is added to TFRecord.
580 r_b6b1.hgu1 46769 is added to TFRecord.
581 r_b6b2.hgu1 46770 is added to TFRecord.
582 r_b6b3.hgu1 46771 is added to TFRecord.
583 r_b6b4.hgu1 46772 is added to TFRecord.
584 r_b6b5.hgu1 46773 is added to TFRecord.
585 r_b6b6.hgu1 46774 is added to TFRecord.
586 r_b6b7.hgu1 46775 is added to TFRecord.
587 r_b6b8.hgu1 46776 is added to TFRecord.
588 r_b6b9.hgu1 46777 is added to TFRecord.
589 r_b6ba.hgu1 46778 is added to TFRecord.
590 r_b6bb.hgu1 46779 is added to TFRecord.
591 r_b6bc.hgu1 46780 is added to TFRecord.
592 r_b6bd.hgu1 46781 is added to TFRecord.
593 r_b6be.hgu1 46782 is added to TFRecord.
594 r_b6bf.hgu1 46783 is added to TFRecord.
595 r_b6c0.hgu1 46784 is added to TFRecord.
596 r_b6c1.hgu1 46785 is added to TFRecord.
597 r_b6c2.hgu1 46786 is added to TFRecord.
598 r_b6c3.hgu1 46787 is added to TFRecord.
599 r_b6c4.hgu1 46788 is added to TFRecord.
600 r_b6c5.hgu1 46789 is added to TFRecord.
601 r_b6c6.hgu1 46790 is added t

767 r_b8b0.hgu1 47280 is added to TFRecord.
768 r_b8b1.hgu1 47281 is added to TFRecord.
769 r_b8b2.hgu1 47282 is added to TFRecord.
770 r_b8b3.hgu1 47283 is added to TFRecord.
771 r_b8b4.hgu1 47284 is added to TFRecord.
772 r_b8b5.hgu1 47285 is added to TFRecord.
773 r_b8b6.hgu1 47286 is added to TFRecord.
774 r_b8b7.hgu1 47287 is added to TFRecord.
775 r_b8b8.hgu1 47288 is added to TFRecord.
776 r_b8b9.hgu1 47289 is added to TFRecord.
777 r_b8ba.hgu1 47290 is added to TFRecord.
778 r_b8bb.hgu1 47291 is added to TFRecord.
779 r_b8bc.hgu1 47292 is added to TFRecord.
780 r_b8bd.hgu1 47293 is added to TFRecord.
781 r_b8be.hgu1 47294 is added to TFRecord.
782 r_b8bf.hgu1 47295 is added to TFRecord.
783 r_b8c0.hgu1 47296 is added to TFRecord.
784 r_b8c1.hgu1 47297 is added to TFRecord.
785 r_b8c2.hgu1 47298 is added to TFRecord.
786 r_b8c3.hgu1 47299 is added to TFRecord.
787 r_b8c4.hgu1 47300 is added to TFRecord.
788 r_b8c5.hgu1 47301 is added to TFRecord.
789 r_b8c6.hgu1 47302 is added t

...

1507 r_c0a4.hgu1 49316 is added to TFRecord.
1508 r_c0a5.hgu1 49317 is added to TFRecord.
1509 r_c0a6.hgu1 49318 is added to TFRecord.
1510 r_c0a7.hgu1 49319 is added to TFRecord.
1511 r_c0a8.hgu1 49320 is added to TFRecord.
1512 r_c0a9.hgu1 49321 is added to TFRecord.
1513 r_c0aa.hgu1 49322 is added to TFRecord.
1514 r_c0ab.hgu1 49323 is added to TFRecord.
1515 r_c0ac.hgu1 49324 is added to TFRecord.
1516 r_c0ad.hgu1 49325 is added to TFRecord.
1517 r_c0ae.hgu1 49326 is added to TFRecord.
1518 r_c0af.hgu1 49327 is added to TFRecord.
1519 r_c0b0.hgu1 49328 is added to TFRecord.
1520 r_c0b1.hgu1 49329 is added to TFRecord.
1521 r_c0b2.hgu1 49330 is added to TFRecord.
1522 r_c0b3.hgu1 49331 is added to TFRecord.
1523 r_c0b4.hgu1 49332 is added to TFRecord.
1524 r_c0b5.hgu1 49333 is added to TFRecord.
1525 r_c0b6.hgu1 49334 is added to TFRecord.
1526 r_c0b7.hgu1 49335 is added to TFRecord.
1527 r_c0b8.hgu1 49336 is added to TFRecord.
1528 r_c0b9.hgu1 49337 is added to TFRecord.
1529 r_c0b

1690 r_c1fd.hgu1 49661 is added to TFRecord.
1691 r_c1fe.hgu1 49662 is added to TFRecord.
1692 r_c2a1.hgu1 49825 is added to TFRecord.
1693 r_c2a2.hgu1 49826 is added to TFRecord.
1694 r_c2a3.hgu1 49827 is added to TFRecord.
1695 r_c2a4.hgu1 49828 is added to TFRecord.
1696 r_c2a5.hgu1 49829 is added to TFRecord.
1697 r_c2a6.hgu1 49830 is added to TFRecord.
1698 r_c2a7.hgu1 49831 is added to TFRecord.
1699 r_c2a8.hgu1 49832 is added to TFRecord.
1700 r_c2a9.hgu1 49833 is added to TFRecord.
1701 r_c2aa.hgu1 49834 is added to TFRecord.
1702 r_c2ab.hgu1 49835 is added to TFRecord.
1703 r_c2ac.hgu1 49836 is added to TFRecord.
1704 r_c2ad.hgu1 49837 is added to TFRecord.
1705 r_c2ae.hgu1 49838 is added to TFRecord.
1706 r_c2af.hgu1 49839 is added to TFRecord.
1707 r_c2b0.hgu1 49840 is added to TFRecord.
1708 r_c2b1.hgu1 49841 is added to TFRecord.
1709 r_c2b2.hgu1 49842 is added to TFRecord.
1710 r_c2b3.hgu1 49843 is added to TFRecord.
1711 r_c2b4.hgu1 49844 is added to TFRecord.
1712 r_c2b

1873 r_c3f8.hgu1 50168 is added to TFRecord.
1874 r_c3f9.hgu1 50169 is added to TFRecord.
1875 r_c3fa.hgu1 50170 is added to TFRecord.
1876 r_c3fb.hgu1 50171 is added to TFRecord.
1877 r_c3fc.hgu1 50172 is added to TFRecord.
1878 r_c3fd.hgu1 50173 is added to TFRecord.
1879 r_c3fe.hgu1 50174 is added to TFRecord.
1880 r_c4a1.hgu1 50337 is added to TFRecord.
1881 r_c4a2.hgu1 50338 is added to TFRecord.
1882 r_c4a3.hgu1 50339 is added to TFRecord.
1883 r_c4a4.hgu1 50340 is added to TFRecord.
1884 r_c4a5.hgu1 50341 is added to TFRecord.
1885 r_c4a6.hgu1 50342 is added to TFRecord.
1886 r_c4a7.hgu1 50343 is added to TFRecord.
1887 r_c4a8.hgu1 50344 is added to TFRecord.
1888 r_c4a9.hgu1 50345 is added to TFRecord.
1889 r_c4aa.hgu1 50346 is added to TFRecord.
1890 r_c4ab.hgu1 50347 is added to TFRecord.
1891 r_c4ac.hgu1 50348 is added to TFRecord.
1892 r_c4ad.hgu1 50349 is added to TFRecord.
1893 r_c4ae.hgu1 50350 is added to TFRecord.
1894 r_c4af.hgu1 50351 is added to TFRecord.
1895 r_c4b

2058 r_c5f5.hgu1 50677 is added to TFRecord.
2059 r_c5f6.hgu1 50678 is added to TFRecord.
2060 r_c5f7.hgu1 50679 is added to TFRecord.
2061 r_c5f8.hgu1 50680 is added to TFRecord.
2062 r_c5f9.hgu1 50681 is added to TFRecord.
2063 r_c5fa.hgu1 50682 is added to TFRecord.
2064 r_c5fb.hgu1 50683 is added to TFRecord.
2065 r_c5fc.hgu1 50684 is added to TFRecord.
2066 r_c5fd.hgu1 50685 is added to TFRecord.
2067 r_c5fe.hgu1 50686 is added to TFRecord.
2068 r_c6a1.hgu1 50849 is added to TFRecord.
2069 r_c6a2.hgu1 50850 is added to TFRecord.
2070 r_c6a3.hgu1 50851 is added to TFRecord.
2071 r_c6a4.hgu1 50852 is added to TFRecord.
2072 r_c6a5.hgu1 50853 is added to TFRecord.
2073 r_c6a6.hgu1 50854 is added to TFRecord.
2074 r_c6a7.hgu1 50855 is added to TFRecord.
2075 r_c6a8.hgu1 50856 is added to TFRecord.
2076 r_c6a9.hgu1 50857 is added to TFRecord.
2077 r_c6aa.hgu1 50858 is added to TFRecord.
2078 r_c6ab.hgu1 50859 is added to TFRecord.
2079 r_c6ac.hgu1 50860 is added to TFRecord.
2080 r_c6a

2245 r_c7f4.hgu1 51188 is added to TFRecord.
2246 r_c7f5.hgu1 51189 is added to TFRecord.
2247 r_c7f6.hgu1 51190 is added to TFRecord.
2248 r_c7f7.hgu1 51191 is added to TFRecord.
2249 r_c7f8.hgu1 51192 is added to TFRecord.
2250 r_c7f9.hgu1 51193 is added to TFRecord.
2251 r_c7fa.hgu1 51194 is added to TFRecord.
2252 r_c7fb.hgu1 51195 is added to TFRecord.
2253 r_c7fc.hgu1 51196 is added to TFRecord.
2254 r_c7fd.hgu1 51197 is added to TFRecord.
2255 r_c7fe.hgu1 51198 is added to TFRecord.
2256 r_c8a1.hgu1 51361 is added to TFRecord.
2257 r_c8a2.hgu1 51362 is added to TFRecord.
2258 r_c8a3.hgu1 51363 is added to TFRecord.
2259 r_c8a4.hgu1 51364 is added to TFRecord.
2260 r_c8a5.hgu1 51365 is added to TFRecord.
2261 r_c8a6.hgu1 51366 is added to TFRecord.
2262 r_c8a7.hgu1 51367 is added to TFRecord.
2263 r_c8a8.hgu1 51368 is added to TFRecord.
2264 r_c8a9.hgu1 51369 is added to TFRecord.
2265 r_c8aa.hgu1 51370 is added to TFRecord.
2266 r_c8ab.hgu1 51371 is added to TFRecord.
2267 r_c8a